In [121]:
import pandas as pd
import numpy as np
import plotly.express as px

In [122]:
%matplotlib inline

In [123]:
df_de = pd.read_csv('D:\\OneDrive\\Vassar\\DataFest\\data\\DE\\de.csv', parse_dates=['DATE'])
df_ca = pd.read_csv('D:\\OneDrive\\Vassar\\DataFest\\data\\CA\\ca.csv', parse_dates=['DATE'])
df_uk = pd.read_csv('D:\\OneDrive\\Vassar\\DataFest\\data\\UK\\uk.csv', parse_dates=['DATE'])
df_us = pd.read_csv('D:\\OneDrive\\Vassar\\DataFest\\data\\US\\US\\us18.csv', parse_dates=['DATE'])

In [124]:
def get_drug_names(df, country):
    # returns a list of drug names for convenience
    if country == 'de':
        i = 14
    elif country == 'ca':
        i = 16
    elif country == 'uk':
        i = 19
    elif country == 'us':
        i = 17
    drugs = [x for x in df.columns if 'NMU' in x][:i]
    drugs = [x.split('_')[0] for x in drugs]
    return drugs

In [136]:
def get_use_cat(df, country):
    # make new df so old dataset is not affected
    new_df = df.copy()
    # get list of drug names
    drugs = get_drug_names(df, country)
    # make categorical variable for each drug
    for drug in drugs:
        new_df[f'{drug}_USE_CAT'] = new_df[f'{drug}_USE'] + new_df[f'{drug}_NMU']
        new_df[f'{drug}_USE_CAT'].fillna(value=0, inplace=True)
    # in the returned df, each drug now has a column indicating how the correspondent uses the drug
    # 0 -> never used
    # 1 -> used for prescription purposes
    # 2 -> used for recreational purposes
    return new_df

In [137]:
def calculate_proportions(df, country):
    new_df = get_use_cat(df)
    drugs = get_drug_names(df, country)
    # empty dict to insert values
    d = {}
    # for each drug, get proportions of recreational use
    for drug in drugs:
        # get number of people for prescription and recreational purposes
        num_pre = new_df[f'{drug}_USE_CAT'].value_counts().loc[1.0]
        num_rec = new_df[f'{drug}_USE_CAT'].value_counts().loc[2.0]
        # get percentage of recreational usage
        percentage = num_rec / (num_pre + num_rec)
        # insert into dictionary
        d[drug] = percentage
    return d

In [138]:
def get_pres_predictors(df, country):
    # returns a df with columns containing prescripted drug use category
    df_use = get_use_cat(df)
    drugs = get_drug_names(df, country)
    pred_cols = [x+'_USE_CAT' for x in drugs]
    return df_use[pred_cols]

In [128]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [129]:
def decision_tree(df):
    X_pres = get_pres_predictors(df_de)
    y = df['DAST_CAT']
    # split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_pres, y, test_size=0.33, random_state=42)
    # train model
    decision_tree = DecisionTreeClassifier(random_state=42, max_depth=2)
    decision_tree = decision_tree.fit(X_train, y_train)
    # add cross-validation???
    cv_score = cross_val_score(decision_tree, X_train, y_train, cv=10).mean()
    print(f'CV-Score: {round(cv_score*100, 4)}%')
    # check perdformance in test set
    df_prediction = pd.DataFrame()
    df_prediction['Predicted'], df_prediction['Actual'] = decision_tree.predict(X_test), y_test.values
    df_prediction = df_prediction.assign(Correct = df_prediction['Predicted'] == df_prediction['Actual'])
    # calculate accuracy in test set
    acc = df_prediction['Correct'].value_counts()[True] / len(df_prediction)
    print(f'Accuracy: {round(acc*100, 4)}%')

In [130]:
decision_tree(df_de)

CV-Score: 52.6673%
Accuracy: 53.4931%


# Can you be a junkie... but unintentionally?

In [131]:
def got_pres_for_pain(df):
    # this function returns a list indicating if a person has received prescription for pain
    df_new = df.copy()
    PRES_FOR_PAIN = []
    for i in range(len(df_new)):
        # Have the patient received prescription for chronic or acute pain?
        PAIN_CHRONIC_RX = df_new.at[i, 'PAIN_CHRONIC_RX']
        PAIN_ACUTE_RX = df_new.at[i, 'PAIN_ACUTE_RX']
        PRES_FOR_PAIN.append(pd.notna(PAIN_CHRONIC_RX) or pd.notna(PAIN_ACUTE_RX))
    return PRES_FOR_PAIN

In [139]:
def get_recr_use(df, country):
    # this function returns a list indicating NMU of one or more drugs
    df_new = df.copy()
    drugs = get_drug_names(df_new, country)
    use_cat = [x+'_USE_CAT' for x in drugs]
    RECR_USE = []
    # loop thru df,
    # if at least one drug has been used for recreational purposes, mark True
    for i in range(len(df)):
        recr_use = False
        for drug in use_cat:
            if df_new.at[i, drug] == 2.0:
                recr_use = True
                break
        RECR_USE.append(recr_use)
    return RECR_USE

In [140]:
def filter_nmu(df, country):
    # this function filters out the people who:
    #   - suffers from chronic or acute pain
    #   - have sought professional help
    #   - have received prescription
    #   - is using one or more prescriptive drug for recreation
    df_new = get_use_cat(df, country)
    # has this person received prescription for pain?
    df_new['PRES_FOR_PAIN'] = got_pres_for_pain(df_new)
    # is this person using the drug for recreational purposes?
    df_new['RECR_USE'] = get_recr_use(df_new, country)
    # drop irrelevant observations
    df_new = df_new.query("PRES_FOR_PAIN==True and RECR_USE==True")
    return df_new

In [147]:
filter_nmu(df_de, 'de')

,DATE,STATUS,QLANG,DEM_GENDER,DEM_AGE,DEM_LOCATION,DEM_POSTAL,DEM_STDNT,DEM_VET,DEM_HEALTH,...,TAP_USE_CAT,SUF_USE_CAT,COD_USE_CAT,DIHY_USE_CAT,HYDM_USE_CAT,STIM_USE_CAT,BENZ_USE_CAT,THC_USE_CAT,PRES_FOR_PAIN,RECR_USE
0,2018-01-12 08:08:01,3,2,1,21,1,786,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True,True
1,2018-01-14 08:08:30,3,2,1,24,1,797,1,0,1,...,1.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,True,True
8,2018-01-11 03:09:52,3,2,1,22,1,793,0,0,0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,True,True
10,2018-01-14 04:13:26,3,2,1,18,1,743,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,True
13,2018-01-14 12:05:27,3,1,1,23,1,688,1,0,0,...,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15031,2017-12-22 08:28:24,3,2,2,63,16,985,0,0,0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,True,True
15042,2018-01-02 07:30:06,3,2,2,65,16,986,0,0,0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,True,True
15043,2018-01-02 05:00:57,3,2,2,57,16,990,0,0,0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,True,True
15044,2018-01-02 04:47:28,3,2,2,63,16,986,0,0,0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,True,True


In [143]:
df_nmu_de = filter_nmu(df_de, 'de')
px.histogram(df_nmu_de, x='DEM_AGE', nbins=20, title='GERMANY')